In [ ]:
import tensorflow as tf
import math
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
print type(mnist.train.images), mnist.train.images.shape
print type(mnist.train.labels), mnist.train.labels.shape

In [ ]:
print type(mnist.validation.images), mnist.validation.images.shape   
print type(mnist.validation.labels), mnist.validation.labels.shape

In [ ]:
print type(mnist.test.images), mnist.test.images.shape
print type(mnist.test.labels), mnist.test.labels.shape

In [ ]:
class VanillaNN:
    def __init__(self):
        self.epoch_list                  = []
        self.train_error_value_list      = []
        self.validation_error_value_list = []
        self.test_error_value_list       = []
        self.test_accuracy_list          = []
            
    def setData(self, n_input, n_classes, trainData, validationData, testData):
        self.n_input     = n_input            # 784
        self.n_classes   = n_classes          # 10
        self.trainData   = trainData
        self.validationData = validationData
        self.testData = testData

    #Create model
    def makeModel(self, n_hidden, learning_rate):
        self.n_hidden        = n_hidden
        self.numHiddenLayer  = len(n_hidden)
        self.learning_rate   = learning_rate
        
        #tf Graph input
        self.x = tf.placeholder(tf.float32, (None, self.n_input))
        self.y = tf.placeholder(tf.float32, (None, self.n_classes))

        if (self.numHiddenLayer == 0):
            self.weight = tf.Variable(tf.zeros([self.n_input, self.n_classes]))
            self.bias = tf.Variable(tf.zeros([self.n_classes]))
            self.pred = tf.add(tf.matmul(self.x, self.weight), self.bias)
        else:
            #Store layers weight & bias
            self.weights = {
                1: tf.Variable(tf.random_normal([self.n_input, self.n_hidden[0]])),
                'out': tf.Variable(tf.random_normal([self.n_hidden[0], self.n_classes]))
            }
            self.biases = {
                1: tf.Variable(tf.random_normal([self.n_hidden[0]])),
                'out': tf.Variable(tf.random_normal([self.n_classes]))
            }
            
            self.u = tf.add(tf.matmul(self.x, self.weights[1]), self.biases[1])
            self.z = tf.nn.relu(self.u)

            if (self.numHiddenLayer >= 2):
                for i in range(self.numHiddenLayer - 1):
                    self.weights[i+2] = tf.Variable(tf.random_normal([self.n_hidden[i], self.n_hidden[i+1]]))
                    self.biases[i+2] = tf.Variable(tf.random_normal([self.n_hidden[i+1]]))
                self.weights['out'] = tf.Variable(tf.random_normal([self.n_hidden[self.numHiddenLayer - 1], self.n_classes]))
                self.biases['out'] = tf.Variable(tf.random_normal([self.n_classes]))
        
                for i in range(self.numHiddenLayer - 1):
                    self.u = tf.add(tf.matmul(self.z, self.weights[i+2]), self.biases[i+2])
                    self.z = tf.nn.relu(self.u)
        
            self.pred = tf.add(tf.matmul(self.z, self.weights['out']), self.biases['out'])
        
        self.error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(self.pred, self.y))
        self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.error)

        self.prediction_ground_truth = tf.equal(tf.argmax(self.pred, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.prediction_ground_truth, tf.float32))
    
    def learning(self, batch_size, training_epochs):
        self.batch_size      = batch_size
        self.training_epochs = training_epochs
        self.init = tf.global_variables_initializer()

        with tf.Session() as sess:
            sess.run(self.init)
            self.total_batch = int(math.ceil(self.trainData.num_examples/float(self.batch_size)))
            print "Total batch: %d" % self.total_batch

            for epoch in range(self.training_epochs):
                for i in range(self.total_batch):
                    batch_images, batch_labels = self.trainData.next_batch(self.batch_size)
                    _, error_value = sess.run((self.optimizer, self.error), 
                                              feed_dict={self.x: batch_images, self.y: batch_labels})

                self.epoch_list.append(epoch)
                
                self.train_error_value_list.append(error_value)

                v_error_value = sess.run(self.error, 
                                         feed_dict={self.x: self.validationData.images, self.y: self.validationData.labels})
                self.validation_error_value_list.append(v_error_value)
                
                self.printLossAccuracyForTestData(epoch, sess)

            self.drawErrorValues()

            self.drawFalsePrediction(sess, 10)

            print("Training % Test finished!")
            
    def printLossAccuracyForTestData(self, epoch, sess):
        accuracy_value, error_value = sess.run((self.accuracy, self.error), 
                                               feed_dict={self.x: self.testData.images, self.y: self.testData.labels})
        self.test_error_value_list.append(error_value)
        self.test_accuracy_list.append(accuracy_value)
        print "epoch: %d, test_error_value: %f, test_accuracy: %f" % (epoch, error_value, accuracy_value)
        
    def drawErrorValues(self):
        fig = plt.figure(figsize=(20, 5))
        plt.subplot(121)
        plt.plot(self.epoch_list, self.train_error_value_list, 'r', label='Train')
        plt.plot(self.epoch_list, self.validation_error_value_list, 'g', label='Validation')
        plt.plot(self.epoch_list, self.test_error_value_list, 'b', label='Test')
        plt.ylabel('Total Error')
        plt.xlabel('Epochs')
        plt.grid(True)
        plt.legend(loc='upper right')
        
        plt.subplot(122)
        plt.plot(self.epoch_list, self.test_accuracy_list, 'b', label='Test')
        plt.ylabel('Accuracy')
        plt.xlabel('Epochs')
        plt.yticks(np.arange(min(self.test_accuracy_list), max(self.test_accuracy_list), 0.05))
        plt.grid(True)
        plt.legend(loc='lower right')            
        plt.show()

    def drawFalsePrediction(self, sess, numPrintImages):
        ground_truth = sess.run(tf.argmax(self.y, 1), feed_dict={self.y: self.testData.labels})
        prediction = sess.run(tf.argmax(self.pred, 1), feed_dict={self.x: self.testData.images})

        fig = plt.figure(figsize=(20, 5))
        j = 1
        for i in range(self.testData.num_examples):
            if (j > numPrintImages):
                break;
            if (prediction[i] != ground_truth[i]):
                print "Error Index: %s, Prediction: %s, Ground Truth: %s" % (i, prediction[i], ground_truth[i])
                img = np.array(self.testData.images[i])
                img.shape = (28, 28)
                plt.subplot(1, numPrintImages, j)
                plt.imshow(img)
                j += 1

## Vanilla (Sigle Layer + Batch)

In [ ]:
#Parameter: Batch_size, Training_epochs, learning_rate, n_hidden
vanilla = VanillaNN()
vanilla.setData(n_input = 784, 
                n_classes = 10, 
                trainData = mnist.train, 
                validationData = mnist.validation, 
                testData = mnist.test)
vanilla.makeModel(n_hidden = [], learning_rate = 0.001)
vanilla.learning(batch_size = 100, training_epochs = 50)

## Vanilla (Multi Layer + Batch) - 2 Hidden Layers, 128 neurons per Hidden Layer 

In [ ]:
vanilla2 = VanillaNN()
vanilla2.setData(n_input = 784,
                      n_classes = 10,
                      trainData = mnist.train,
                      validationData = mnist.validation,
                      testData = mnist.test)
vanilla2.makeModel(n_hidden = [128, 128], learning_rate = 0.001)
vanilla2.learning(batch_size = 100, training_epochs = 50)

## Vanilla (Multi Layer + Batch) - 3 Hidden Layers, 256 neurons per Hidden Layer 

In [ ]:
#Parameter: Batch_size, Training_epochs, learning_rate, n_hidden
vanilla3 = VanillaNN()
vanilla3.setData(n_input = 784,
                 n_classes = 10,
                 trainData = mnist.train,
                 validationData = mnist.validation,
                 testData = mnist.test)
vanilla3.makeModel(n_hidden = [256, 256, 256], learning_rate = 0.001)
vanilla3.learning(batch_size = 100, training_epochs = 50)

In [ ]:
plt.plot(vanilla.epoch_list, vanilla.test_accuracy_list, 'r', label='Vanilla - Sigle Layer')
plt.plot(vanilla2.epoch_list, vanilla2.test_accuracy_list, 'g', label='Vanilla2 - 128 neurons/2 Hidden Layers')
plt.plot(vanilla3.epoch_list, vanilla3.test_accuracy_list, 'b', label='Vanilla3 - 256 neurons/3 Hidden Layers')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.grid(True)
plt.legend(loc='lower right')            
plt.show()